# Testing for unit roots in a time series 

##### Jorge De León 
##### Universidad del Rosario


### We extract the data from a PDF and convert it into a .xlsx 

In [216]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.linear_model import LinearRegression
import seaborn
from arch.unitroot import PhillipsPerron
from sklearn.metrics import r2_score

In [55]:
velocity = pd.read_excel("velocity.xlsx")
velocity['time'] = velocity.index + 1


### We have these 3 models:

- $y_{t} = \gamma _{1}y_{t-1} + \epsilon _{t}$ Random walk
- $y_{t} = \gamma_{0} + \gamma _{1}y_{t-1} + \epsilon _{t}$ Randon walk with a drift
- $y_{t} = \gamma_{0} + \gamma _{1}y_{t-1} + \gamma_{2} t +\epsilon _{t}$ randon walk with a drift and time trend

Now we estimate those regressions

In [59]:
mod1 = smf.ols(formula = 'velocity ~ 0 + velocity.shift(1)', data=velocity).fit()
mod2 = smf.ols(formula = 'velocity ~ velocity.shift(1)', data=velocity).fit()
mod3 = smf.ols(formula = 'velocity ~ velocity.shift(1) + time', data=velocity).fit()

# Can use the following metod to see the results of any of the models above 
mod3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               velocity   R-squared:                       0.966
Model:                            OLS   Adj. R-squared:                  0.966
Method:                 Least Squares   F-statistic:                     1268.
Date:                Wed, 22 Feb 2023   Prob (F-statistic):           1.31e-65
Time:                        12:34:57   Log-Likelihood:                 27.973
No. Observations:                  91   AIC:                            -49.95
Df Residuals:                      88   BIC:                            -42.41
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             0.2661      0.188      1.414      0.161      -0.108       0.640
velocity.shift(1)     0.9143      0.046     20.084      0.000       0.824       1.005
time                 -0.0019      0.002     -1.107      0.271      -0.005       0.002
==============================================================================
Omnibus:                        8.315   Durbin-Watson:                   2.045
Prob(Omnibus):                  0.016   Jarque-Bera (JB):               16.457
Skew:                          -0.157   Prob(JB):                     0.000267
Kurtosis:                       5.059   Cond. No.                         549.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Phillips Perron test for Unit Roots 
 - pp1: model with no time trend and no constant
 - pp2: model with constant 
 - pp3: model wit time trend and a constant


 Note: Every model uses just 1 lag. 

In [66]:
pp1 = PhillipsPerron(velocity["velocity"], lags = 1,  trend = "n")
pp2 = PhillipsPerron(velocity["velocity"], lags = 1,  trend = "c")
pp3 = PhillipsPerron(velocity["velocity"], lags = 1,  trend = "ct")

# To see the result:
print(pp3)

     Phillips-Perron Test (Z-tau)    
Test Statistic                 -1.835
P-value                         0.688
Lags                                1
-------------------------------------

Trend: Constant and Linear Time Trend
Critical Values: -4.06 (1%), -3.46 (5%), -3.16 (10%)
Null Hypothesis: The process contains a unit root.
Alternative Hypothesis: The process is weakly stationary.


## Montecarlo simulation 

- $Y_{t} = \alpha + \beta t + \hat{u_{t}}$

1000 obs and 1000 rep 

In [257]:
np.random.seed(12345) #Seed 
repeticiones = 1000
Obs = 100
r1 = []
r2 = []
r3 = []

residuals = []
residuals1 = []
alpha = 0
beta = 0
A = 0
be = 0

betas_gorro = []
alphas_gorro = []
R_scuared = []


In [254]:
# Now we replace Y with the formmula of the model 

for z in range(repeticiones):
    Y = []
    X = []
    alpha = 0
    beta = 0
    Obs = 100
    for i in range(Obs):
        Y.append(0)
        X.append(i+1)
    
    for i in range(Obs):
        u  = np.random.normal(loc=0, scale=1)
        if i == 0:
            Y[i] = u
        else:
            Y[i] = Y[i-1] + u

    dic = {'Y' : Y, 'X': X}
    panel = pd.DataFrame(data=dic) #Data Frame
    y = panel["Y"].values.reshape(-1, 1)
    x = panel["X"].values.reshape(-1, 1)

    reg1 = LinearRegression().fit(x, y)
    #R_scuared.append(float(reg1.score(x,y)))
    preds = reg1.predict(x)
    R_scuared.append(r2_score(y, preds))

    residuals = y - preds

    residuals1 = sm.tsa.acf(residuals)
    r1.append(residuals1[1])
    r2.append(residuals1[2])
    r3.append(residuals1[3])



### Datos replicados del paper

In [256]:
print(np.mean(R_scuared))
print(np.mean(r1))
print(np.mean(r2))
print(np.mean(r3))

0.4415277141440973
0.8798132075848984
0.7696503567359734
0.6689538175809225


### Modelo de 1000 repeticiones con 20 observaciones

In [258]:
for z in range(repeticiones):
    Y = []
    X = []
    alpha = 0
    beta = 0
    Obs = 20
    for i in range(Obs):
        Y.append(0)
        X.append(i+1)
    
    for i in range(Obs):
        u  = np.random.normal(loc=0, scale=1)
        if i == 0:
            Y[i] = u
        else:
            Y[i] = Y[i-1] + u

    dic = {'Y' : Y, 'X': X}
    panel = pd.DataFrame(data=dic) #Data Frame
    y = panel["Y"].values.reshape(-1, 1)
    x = panel["X"].values.reshape(-1, 1)

    reg1 = LinearRegression().fit(x, y)
    #R_scuared.append(float(reg1.score(x,y)))
    preds = reg1.predict(x)
    R_scuared.append(r2_score(y, preds))

    residuals = y - preds

    residuals1 = sm.tsa.acf(residuals)
    r1.append(residuals1[1])
    r2.append(residuals1[2])
    r3.append(residuals1[3])

#### We obtain the following parameters

In [259]:
print(np.mean(R_scuared))
print(np.mean(r1))
print(np.mean(r2))
print(np.mean(r3))

0.4222633620007054
0.4858723618135954
0.16460345862813203
-0.043182226065671764
